In [1]:
from pybullet_environment import *
from guide import *

pybullet build time: Jul 17 2023 13:38:55


In [2]:
env = RobotEnvironment(manipulator = True)
guide = IntersectionVolumeGuide(env)

argv[0]=
startThreads creating 1 threads.
starting thread 0
started thread 0 
argc=3
argv[0] = --unused
argv[1] = 
argv[2] = --start_demo_name=Physics Server
ExampleBrowserThreadFunc started
X11 functions dynamically loaded using dlopen/dlsym OK!
X11 functions dynamically loaded using dlopen/dlsym OK!
Creating context
Created GL 3.3 context
Direct GLX rendering context obtained
Making context current
GL_VENDOR=NVIDIA Corporation
GL_RENDERER=NVIDIA GeForce GTX 1660 Ti/PCIe/SSE2
GL_VERSION=3.3.0 NVIDIA 535.86.05
GL_SHADING_LANGUAGE_VERSION=3.30 NVIDIA via Cg compiler
pthread_getconcurrency()=0
Version = 3.3.0 NVIDIA 535.86.05
Vendor = NVIDIA Corporation
Renderer = NVIDIA GeForce GTX 1660 Ti/PCIe/SSE2
b3Printf: Selected demo: Physics Server
startThreads creating 1 threads.
starting thread 0
started thread 0 
MotionThreadFunc thread started


: 

In [3]:
env.draw_link_bounding_boxes()

In [4]:
env.clear_bounding_boxes()

In [3]:
joint_config = np.zeros((9,))[:-2]

In [24]:
B_pos = np.array([0.21, 0.11, 0.51])
B_ori = np.array([0.1, 0.3, -0.4])
B_quat = np.array(env.client_id.getQuaternionFromEuler(B_ori))
B_pose = np.array([*B_pos, *B_quat])

B_size = np.array([0.15, 0.3, 0.08])

: 

In [23]:
link_id = 8
joint_config = torch.tensor(joint_config)
joint_config.requires_grad = True

link_dims = guide.link_dimensions[link_id]
link_transform = guide.get_link_transform(link_id, joint_config)

print(link_transform)
print(link_transform.requires_grad)

cost = torch.sum(link_transform)
cost.backward()

print(joint_config.grad)

# link_quat = env.rotation_matrix_to_quaternion(link_transform[:3, :3])
# link_pos = link_transform[:3, -1]

# vuid = env.client_id.createVisualShape(p.GEOM_BOX, 
#                                        halfExtents = link_dims/2,
#                                        rgbaColor = np.hstack([env.colors['red'], np.array([1.0])]))

# link_id = env.client_id.createMultiBody(baseVisualShapeIndex = vuid, 
#                                        basePosition = link_pos, 
#                                        baseOrientation = link_quat)

# print(np.array(env.client_id.getMatrixFromQuaternion(link_quat)).reshape((3, 3)))
# print(link_transform[:3, :3])

tensor([[-0.5698,  0.6872,  0.4506,  0.4154],
        [ 0.8144,  0.3990,  0.4214, -0.3780],
        [ 0.1098,  0.6070, -0.7871,  0.5810],
        [ 0.0000,  0.0000,  0.0000,  1.0000]], grad_fn=<MmBackward0>)
True
tensor([-0.2734,  1.2932,  0.2932,  1.4777,  1.1878,  1.3793,  1.3343],
       dtype=torch.float64)


/tmp/ipykernel_17086/872007494.py:2: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  joint_config = torch.tensor(joint_config)


In [ ]:
for i in range(10, 1, -1):
    env.client_id.removeBody(i)

In [6]:
# Joint 1 => Link 0
link_id = 1
joint_id = 2

link_transform = env.pose_to_transformation(env.link_poses[link_id])
fk = (guide.forward_kinematics(joint_id, torch.tensor(joint_config))).numpy()
fk_inv = env.inverse_of_transform(fk)

# print(link_transform)

x = fk_inv @ link_transform
print(x)
# print(link_transform)
# print(fk @ x)

[[ 1.00000000e+00  0.00000000e+00  0.00000000e+00 -8.42500000e-05]
 [ 0.00000000e+00  1.00000000e+00 -4.37113883e-08 -6.93950016e-02]
 [ 0.00000000e+00  4.37113883e-08  1.00000000e+00  3.71961970e-02]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]


In [7]:
x_prev = x.copy()
print(x_prev)

[[ 1.00000000e+00  0.00000000e+00  0.00000000e+00 -8.42500000e-05]
 [ 0.00000000e+00  1.00000000e+00 -4.37113883e-08 -6.93950016e-02]
 [ 0.00000000e+00  4.37113883e-08  1.00000000e+00  3.71961970e-02]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]


In [8]:
t = guide.get_link_transform(1, torch.tensor(joint_config))
print(t)
print(link_transform)

tensor([[ 1.0000e+00,  0.0000e+00,  0.0000e+00, -8.4250e-05],
        [ 0.0000e+00, -4.3711e-08,  1.0000e+00,  3.7196e-02],
        [ 0.0000e+00, -1.0000e+00, -4.3711e-08,  4.0240e-01],
        [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  1.0000e+00]])
[[ 1.00000000e+00 -0.00000000e+00  0.00000000e+00 -8.42500000e-05]
 [ 0.00000000e+00  0.00000000e+00  1.00000000e+00  3.71962000e-02]
 [-0.00000000e+00 -1.00000000e+00  0.00000000e+00  4.02395004e-01]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]


X connection to :1 broken (explicit kill or server shutdown).


: 

In [104]:
joint_frame_id = env.draw_frame(fk)

In [105]:
link_frame_id = env.draw_frame(link_transform)

In [59]:
env.remove_frame(joint_frame_id)
env.remove_frame(link_frame_id)

In [112]:
env.client_id.removeAllUserDebugItems()

In [21]:
joint_config = np.array([-1.9300085,  1.3141115,  2.1577177, -1.5229155, -0.7427585, 1.4192199, -1.3984755])

for i, joint_ind in enumerate(env.joints):
    env.client_id.resetJointState(env.manipulator, joint_ind, joint_config[i])

In [28]:
joint_config = np.zeros((9,))[:-2]
joint_config[0] = np.pi/4

for i, joint_ind in enumerate(env.joints):
    env.client_id.resetJointState(env.manipulator, joint_ind, joint_config[i])

In [146]:
joint_config = np.zeros((9,))[:-2]

for i, joint_ind in enumerate(env.joints):
    env.client_id.resetJointState(env.manipulator, joint_ind, joint_config[i])